In [18]:
!pip install tokenizer

In [23]:
!pip install keras 

In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('data.csv')

In [3]:
df.head()

,Unnamed: 0,X,ID,Title,Desc,Area.Path,Category
0,1,28,450061,Re-apply workaround for Heating Station Functi...,Description:\r\nThe Stop Regulate command spor...,\GPS\NewGenSW\ArchDecomp\AS\DSS,0
1,2,33,450673,DSS has a to short timeout for the Plate Handl...,Descritpion:\r\nDSS has a to short timeout (18...,\GPS\NewGenSW\ArchDecomp\AS\DSS,0
2,3,88,458264,Timeout for ProcessHead back Teach is to short...,Timeout for ProcessHead back Teach is to short...,\GPS\NewGenSW\ArchDecomp\AS\DSS,0
3,4,124,464061,Sample Head Tip Handling Check in Operations -...,Description:\r\nThe Sample Head Tip Handling C...,\GPS\NewGenSW\ArchDecomp\AS\DSS,0
4,5,125,464062,Text of Sample Cell RunIn Loop description not...,Description: \r\nThe following text needs to b...,\GPS\NewGenSW\ArchDecomp\AS\DSS,0


In [4]:
df.Category.unique()

array([0, 1, 4, 5], dtype=int64)

In [5]:
df.Desc[1]

'Descritpion:\r\nDSS has a to short timeout (180sec)  for the Plate Handler P teach (OQ List), but the Plate Handler needs more.\r\n\r\nno log are are needed\r\n\r\n\r\n\r\nSteps to reproduce:\r\nStart a PlateHandlerP teach with DSS version 01.01.04.1005 from the oq list\r\n\r\n\r\nexpected:\r\nthat the default time out of the teach commando will be used (default timeout is 600sec)\r\n'

In [6]:
df.shape

(846, 7)

In [7]:
df_new = pd.DataFrame()

In [8]:
df_new['desc'] = df.Desc

In [9]:
df_new['category'] = df.Category

In [10]:
df_new.head()

,desc,category
0,Description:\r\nThe Stop Regulate command spor...,0
1,Descritpion:\r\nDSS has a to short timeout (18...,0
2,Timeout for ProcessHead back Teach is to short...,0
3,Description:\r\nThe Sample Head Tip Handling C...,0
4,Description: \r\nThe following text needs to b...,0


In [11]:
X = df_new.desc
df_new.desc.head()

0    Description:\r\nThe Stop Regulate command spor...
1    Descritpion:\r\nDSS has a to short timeout (18...
2    Timeout for ProcessHead back Teach is to short...
3    Description:\r\nThe Sample Head Tip Handling C...
4    Description: \r\nThe following text needs to b...
Name: desc, dtype: object

In [12]:
import re
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
#df_new.desc = df_new.desc.apply(lambda x: x.lower())
df_new.desc = df_new.desc.apply(lambda x: re.sub("[!@#$+%*:()'-]", ' ', x))
#df.text = df.text.apply(lambda x: re.sub(r'\W+', '', x))
print(df_new.desc.head())
df_new.desc = df_new.desc.apply(lambda x:[word for word in x.split() if word not in stop_words])
#print(df_new.desc.head())

0    Description \r\nThe Stop Regulate command spor...
1    Descritpion \r\nDSS has a to short timeout  18...
2    Timeout for ProcessHead back Teach is to short...
3    Description \r\nThe Sample Head Tip Handling C...
4    Description  \r\nThe following text needs to b...
Name: desc, dtype: object


In [13]:
df_new['desc'] = df_new['desc'].apply(', '.join)

In [14]:
print(df_new.desc.head())
type(df_new.desc[1])

0    Description, The, Stop, Regulate, command, spo...
1    Descritpion, DSS, short, timeout, 180sec, Plat...
2    Timeout, ProcessHead, back, Teach, short, DSS,...
3    Description, The, Sample, Head, Tip, Handling,...
4    Description, The, following, text, needs, remo...
Name: desc, dtype: object


str

### using autoencoder

In [24]:
#from tokenizer import tokenize

import keras
from keras.preprocessing.text import Tokenizer
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df_new['desc'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "C:\Users\arpita.c.gupta\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 18, in swig_import_helper
    fp, pathname, description = imp.find_module('_pywrap_tensorflow_internal', [dirname(__file__)])
  File "C:\Users\arpita.c.gupta\AppData\Local\Continuum\anaconda3\lib\imp.py", line 296, in find_module
    raise ImportError(_ERR_MSG.format(name), name=name)
ImportError: No module named '_pywrap_tensorflow_internal'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\arpita.c.gupta\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\arpita.c.gupta\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>

In [16]:
import tokenize
X = tokenizer.texts_to_sequences(df_new['desc'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

NameError: name 'tokenizer' is not defined

In [25]:
#### https://realpython.com/python-keras-text-classification/
#### http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/
#### https://blog.keras.io/building-autoencoders-in-keras.html